In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9172
2,Huelva,Confirmados PDIA 14 días,2102
3,Huelva,Tasa PDIA 14 días,"411,5572802759901"
4,Huelva,Confirmados PDIA 7 días,847
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,21
629,Municipio de Huelva sin especificar,Total Confirmados,125
630,Municipio de Huelva sin especificar,Curados,55


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9172.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2574.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 568 personas en los últimos 7 días 

Un positivo PCR cada 228 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9172.0,847.0,2102.0,510743.0,165.836830,411.557280,171.0
Huelva-Costa,4996.0,557.0,1277.0,288115.0,193.325582,443.225795,114.0
Huelva (capital),2574.0,253.0,629.0,143663.0,176.106583,437.830200,57.0
Condado-Campiña,2838.0,211.0,630.0,155057.0,136.078990,406.302199,38.0
Lepe,795.0,104.0,211.0,27431.0,379.133098,769.202727,22.0
Ayamonte,286.0,65.0,125.0,20946.0,310.321780,596.772653,18.0
Sierra de Huelva-Andévalo Central,1214.0,58.0,172.0,67571.0,85.835640,254.547069,12.0
Moguer,269.0,46.0,99.0,22088.0,208.257878,448.207171,11.0
Aljaraque,336.0,36.0,71.0,21260.0,169.332079,333.960489,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,17.0,3.0,14.0,548.0,547.445255,2554.744526,0.0
Lucena del Puerto,60.0,14.0,33.0,3371.0,415.307031,978.938001,2.0
Palma del Condado (La),410.0,29.0,95.0,10761.0,269.491683,882.817582,8.0
Lepe,795.0,104.0,211.0,27431.0,379.133098,769.202727,22.0
Niebla,98.0,13.0,28.0,4117.0,315.763906,680.106874,0.0
Minas de Riotinto,62.0,10.0,26.0,3848.0,259.875260,675.675676,1.0
Rociana del Condado,99.0,21.0,51.0,7855.0,267.345640,649.267982,3.0
Villarrasa,90.0,3.0,14.0,2176.0,137.867647,643.382353,2.0
Ayamonte,286.0,65.0,125.0,20946.0,310.321780,596.772653,18.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Trigueros,124.0,1.0,16.0,7713.0,12.965124,207.441981,2.0,0.062500
Puebla de Guzmán,21.0,1.0,12.0,3073.0,32.541490,390.497885,1.0,0.083333
San Juan del Puerto,133.0,2.0,17.0,9300.0,21.505376,182.795699,2.0,0.117647
Almonte,424.0,11.0,91.0,24191.0,45.471456,376.172957,1.0,0.120879
Villablanca,54.0,2.0,14.0,2848.0,70.224719,491.573034,0.0,0.142857
Valverde del Camino,258.0,5.0,30.0,12820.0,39.001560,234.009360,1.0,0.166667
Paterna del Campo,50.0,1.0,6.0,3509.0,28.498148,170.988886,0.0,0.166667
Beas,50.0,3.0,17.0,4257.0,70.472163,399.342260,0.0,0.176471
Aracena,280.0,4.0,22.0,8107.0,49.340076,271.370421,0.0,0.181818
